In [1]:
from scipy.linalg import null_space
#\section*{Введение}
#12 - 26

#Пояснения

Давайте последовательно разберёмся в каждом из вопросов:
1. **Что такое предобусловленность?**
   Предобусловленность — это процедура, применяемая к системе 
   уравнений для улучшения её численных свойств. Суть метода 
   заключается в умножении исходной системы уравнений на так 
   называемый "предобусловитель" с целью сделать новую систему 
   более "хорошо обусловленной" или, другими словами, 
   уменьшить число обусловленности матрицы системы. 
   В результате, итерационные методы могут сходиться быстрее 
   к решению новой системы, чем к решению исходной.
2. **Что такое седловая задача?**
   Седловые задачи — это задачи оптимизации, где функционал 
   имеет как минимумы, так и максимумы в различных областях 
   пространства решений. Графически такая функция напоминает 
   седло коня, отсюда и название. В контексте систем линейных
     уравнений седловая задача часто связана с системами, 
     имеющими матрицу, которая не является ни положительно 
     определенной, ни отрицательно определенной.
3. **Что считается "большой размерностью"?**
   "Большая размерность" — 10^4 10^6 10^9 уравнений
4. **Что такое подпространства Крылова?**
   Подпространство Крылова — это последовательность векторных 
   пространств, генерируемых векторами вида 
   \( b, Ab, A^2b, \ldots, A^kb \), 
   где \( A \) — матрица, 
   а \( b \) — вектор. 
   Эти пространства играют ключевую роль в итерационных 
   методах для решения систем линейных уравнений, таких 
   как методы Крылова.
5. **Что за метод GMRES?**
   GMRES (Generalized Minimal RESidual) — это итерационный 
   метод решения систем линейных уравнений. Он основан на 
   поиске решения в подпространствах Крылова таким образом, 
   чтобы минимизировать норму невязки. GMRES является одним 
   из наиболее популярных методов решения несимм

\[ Av = b, \quad b \in \mathbb{C}^n, \]
где \( A \in \mathbb{C}^{n \times n} \) — 
нерегулярно положительно определенная матрица.

Для проверки того, является ли матрица положительно 
определенной, можно использовать различные критерии.
 Одним из наиболее распространенных методов является 
 проверка собственных значений матрицы: матрица является 
 положительно определенной тогда и только тогда, когда 
 все её собственные значения строго положительны.

In [ ]:
import numpy as np

def is_positive_definite(A):
    eigenvalues = np.linalg.eigvals(A)
    return np.all(eigenvalues > 0)

# Размерность матрицы и вектора
n = 10**2

\[ Av = b, \quad b \in \mathbb{C}^n, \]

In [ ]:

# Генерация случайной матрицы
# Создаем функцию для генерации случайной положительно определенной матрицы
def generate_positive_definite_matrix(n):
    # Генерация случайной матрицы
    M = np.random.randn(n, n) + 1j * np.random.randn(n, n)
    # Преобразование матрицы в положительно определенную
    return M @ np.conj(M.T)

# Генерация положительно определенной матрицы
A = generate_positive_definite_matrix(n)

# Проверка
is_pos_def = is_positive_definite(A)

# Возвращаем результат проверки и первые несколько элементов для демонстрации
A_sample = A[:5, :5]
print(is_pos_def)

# Генерация случайного вектора
b = np.random.randn(n) + 1j * np.random.randn(n)

# Возвращаем первые несколько элементов для демонстрации
b_sample = b[:5]

print(A_sample, b_sample)

#24

def condition_number(matrix):
    """Вычисляет число обусловленности матрицы."""
    return np.linalg.cond(matrix)

def improve_conditioning(A, B):
    """Проверяет, является ли матрица B^-1*A лучше обусловленной, чем A."""
    original_cond = condition_number(A)
    preconditioned_cond = condition_number(np.linalg.inv(B) @ A)
    
    return original_cond, preconditioned_cond, preconditioned_cond < original_cond

# Возьмем пример с меньшей размерностью из-за ограничений по памяти
n_small = 500

# Генерация случайной матрицы A и предобусловителя B
A_small = np.random.randn(n_small, n_small) + 1j * np.random.randn(n_small, n_small)
B_small = generate_positive_definite_matrix(n_small)

original_cond, preconditioned_cond, is_better = improve_conditioning(A_small, B_small)

print(original_cond, preconditioned_cond, is_better)

#B^-1*A
#имеет наибольшое число собственных значений на периферии своего спекта

# Вычисление собственных значений матрицы B^-1*A
eigenvalues = np.linalg.eigvals(np.linalg.inv(B_small) @ A_small)

# Определение радиуса спектра (расстояния от начала координат до наиболее удаленного собственного значения)
spectral_radius = np.max(np.abs(eigenvalues))

# Определение периферийных собственных значений (те, которые близки к радиусу спектра)
tolerance = 0.01  # допустимая погрешность для определения "близости"
peripheral_eigenvalues = eigenvalues[np.abs(np.abs(eigenvalues) - spectral_radius) < tolerance]

print(spectral_radius, peripheral_eigenvalues)

#Второй лист
#\section{Итерационные методы решения СЛАУ с сильно 

Матрицу \(A\) системы (1) представим в виде суммы её 
диагонали и косоэрмитовой части:
\[ A = A_0 + A_1, \]
где
\[ A_0 = \frac{1}{2}(A + A^*), 
\quad A_1 = \frac{1}{2}(A - A^*). \] (3)

Что такое косоэрмитовая часть ?
Что такое A^*

1. **Косоэрмитова часть матрицы**:
   Косоэрмитова (или антиэрмитова) часть матрицы — это 
   часть матрицы, которая остается после вычитания её 
   эрмитовой (или сопряженно-транспонированной) части.
   Другими словами, это часть матрицы, которая "отвечает" 
   за асимметричность в отношении её эрмитовой части.

2. **\(A^*\)**:
   Символ \(A^*\) обозначает эрмитово сопряжение матрицы 
   \(A\). Это комбинация транспонирования матрицы (замена 
   строк на столбцы) и взятия комплексного сопряжения каждого 
   элемента. Для реальных матриц эрмитово сопряжение 
   эквивалентно обычному транспонированию.

В формулах, которые вы предоставили:
- \(A_0\) представляет собой эрмитову часть матрицы \(A\), 
которая является симметричной относительно своей главной 
диагонали.
- \(A_1\) представляет собой косоэрмитову часть матрицы \(A\), 
которая является асимметричной относительно своей главной 
диагонали.